In [1]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
import os
load_dotenv()
PROJECT_ROOT = os.getenv('PROJECT_ROOT')

# Add PROJECT_ROOT to the Python path
import sys
sys.path.append(PROJECT_ROOT)


# Airports

In [2]:
import pandas as pd
airports_df = pd.read_csv(os.path.join(PROJECT_ROOT, "data", "airac", "airports.csv"))

In [ ]:
airports_df.head()

# Base ATS Graph

In [ ]:
import networkx as nx
ats_graph = nx.read_graphml(os.path.join(PROJECT_ROOT, "data", "graphs", "ats_graph.graphml"))
print(f'ATS graph loaded with {len(ats_graph.nodes())} nodes and {len(ats_graph.edges())} edges')

# Free Route Airspace (FRA) Assimilation

In [ ]:
fra_df = pd.read_csv(os.path.join(PROJECT_ROOT, "data", "rad", "FRA_PTS.csv"))
fra_df.head()

In [6]:
def convert_coord_for_fra_df(coord_str):
        """Convert coordinates from format like 'N404519' or 'E0183830' to decimal degrees"""
        try:
            direction = coord_str[0]
            degrees = float(coord_str[1:-4])
            decimals = float(coord_str[-4:]) / 10000
            decimal = round(degrees + decimals, 4)
            return decimal if direction in ['N', 'E'] else -decimal
        except (ValueError, IndexError):
            return None

In [16]:
import random
from utils.haversine import haversine_distance

def generate_random_two_digits():
    return str(random.randint(0, 99)).zfill(2)

def add_node_with_refs(G, node_id, lat, lon, type='FRA'):
    if node_id in G.nodes:
        # Fix already exists, check if coordinates match
        if abs(G.nodes[node_id]['lat'] - lat) > 1e-4 or abs(G.nodes[node_id]['lon'] - lon) > 1e-4:
            # print(f"Fix {node_id} has different coordinates: {G.nodes[node_id]['lat']}, {G.nodes[node_id]['lon']} != {lat}, {lon}")
            original_node_id = node_id
            node_id = node_id + '_' + generate_random_two_digits()
            G.add_node(node_id, lat=lat, lon=lon, type=type, refs='')
            # Modify the refs of the original node
            
            if 'refs' in G.nodes[original_node_id]:
                original_node_id_refs = G.nodes[original_node_id]['refs']
            else: 
                original_node_id_refs = ''
                
            G.nodes[original_node_id]['refs'] = original_node_id_refs + f'{node_id}, '
        return node_id
    else:
        G.add_node(node_id, lat=lat, lon=lon, type=type, refs='')
        return node_id
    
def create_edge_with_closest_refs(graph_ats, from_fix, to_fix, threshold = 100, type='DCT'):
    if graph_ats.has_edge(from_fix, to_fix):
        return
    from_fix_lat = graph_ats.nodes[from_fix]['lat']
    from_fix_lon = graph_ats.nodes[from_fix]['lon']
    to_fix_lat = graph_ats.nodes[to_fix]['lat']
    to_fix_lon = graph_ats.nodes[to_fix]['lon']
    distance = haversine_distance(from_fix_lat, from_fix_lon, to_fix_lat, to_fix_lon)
    
    # print(f'DCT link between {from_fix} and {to_fix} is {distance:.2f}nm. Considering alternatives...')
    if distance < threshold:
        if not graph_ats.has_edge(from_fix, to_fix):
            graph_ats.add_edge(from_fix, to_fix,
                            distance=distance,
                            airway='',
                            edge_type=type)
        return # do nothing if the distance is less than the threshold
    
    if 'refs' in graph_ats.nodes[from_fix]:
        from_fix_refs = graph_ats.nodes[from_fix]['refs']
    else:
        from_fix_refs = ''
    if 'refs' in graph_ats.nodes[to_fix]:
        to_fix_refs = graph_ats.nodes[to_fix]['refs']
    else:
        to_fix_refs = ''
    # Split refs strings into lists
    from_fix_refs = from_fix_refs.split(',') if from_fix_refs else []
    to_fix_refs = to_fix_refs.split(',') if to_fix_refs else []
    if len(from_fix_refs) == 0 and len(to_fix_refs) == 0:
        # No refs, add the edge directly
        if not graph_ats.has_edge(from_fix, to_fix):
            graph_ats.add_edge(from_fix, to_fix,
                            distance=distance,
                            airway='',
                            edge_type=type)
        return
    # Add the current fixes to the refs
    from_fix_refs.append(from_fix)
    to_fix_refs.append(to_fix)
    
    # Find the closest pair of refs
    min_distance = float('inf')
    best_from_ref = None 
    best_to_ref = None
    
    for from_ref in from_fix_refs:
        if from_ref not in graph_ats.nodes:
            continue
        for to_ref in to_fix_refs:
            if to_ref not in graph_ats.nodes:
                continue
            # Get coordinates
            from_ref_lat = graph_ats.nodes[from_ref]['lat']
            from_ref_lon = graph_ats.nodes[from_ref]['lon']
            to_ref_lat = graph_ats.nodes[to_ref]['lat']
            to_ref_lon = graph_ats.nodes[to_ref]['lon']
            
            # Calculate simple distance (absolute difference)
            dist = abs(from_ref_lat - to_ref_lat) + abs(from_ref_lon - to_ref_lon)
            
            if dist < min_distance:
                min_distance = dist
                best_from_ref = from_ref
                best_to_ref = to_ref

    # Recalculate the distance in nm
    best_min_distance = haversine_distance(
        graph_ats.nodes[best_from_ref]['lat'], 
        graph_ats.nodes[best_from_ref]['lon'], 
        graph_ats.nodes[best_to_ref]['lat'], 
        graph_ats.nodes[best_to_ref]['lon']
    )
    
    # Create edge between closest refs if found
    if best_from_ref and best_to_ref:
        if not graph_ats.has_edge(best_from_ref, best_to_ref):
            graph_ats.add_edge(best_from_ref, best_to_ref,
                            distance=best_min_distance,
                            airway='',
                            edge_type=type)
            # print(f'Established link between {best_from_ref} and {best_to_ref} instead. New distance is {best_min_distance}')

    return

In [17]:
from tqdm import tqdm

def load_fra_graph_into_ats_graph(ats_graph, fra_df = None, fra_df_path = None):
    
    ats_graph = ats_graph.copy()
    fra_df = fra_df.copy()

    print(f'Building FRA routing options...')
    # Load the FRA graph from a GraphML file
    if fra_df is None:
        if fra_df_path is None:
            raise ValueError("Either fra_graph or fra_graph_path must be provided")
        fra_df = pd.read_csv(fra_df_path)
        # columns of fra_df: chg_rec,pt_type,fra_pt,fra_lat,fra_lon,fra_name,fra_rel_enroute,fra_rel_arr_dep,arr_apt,dep_apt,flos,lvl_avail,time_avail,loc_ind,rmk

    # Uppercase and strip the fra_name column to standardize the names
    fra_df['fra_name'] = fra_df['fra_name'].str.upper().str.strip()

    # Convert fra_lat and fra_lon to decimal degrees
    fra_df['fra_lat'] = fra_df['fra_lat'].apply(convert_coord_for_fra_df)
    fra_df['fra_lon'] = fra_df['fra_lon'].apply(convert_coord_for_fra_df)
    
    fra_names = fra_df['fra_name'].unique()

    for fra in fra_names:
        fra_df_subset = fra_df[fra_df['fra_name'] == fra]
        # Add all FRA points from this subset to the graph if they don't exist
        fra_map = {} # stores mapping ABADI -> ABADI_01 if they are not unique
        for _, row in fra_df_subset.iterrows():
            node_id = row['fra_pt']
            renamed_id = add_node_with_refs(ats_graph, node_id, row['fra_lat'], row['fra_lon'], type='FRA')
            if renamed_id != node_id:
                fra_map[node_id] = renamed_id

        print(f'{len(fra_map)} FRA points renamed for {fra}')

        # Get all FRA points for this name and their types
        fra_points = [(row['fra_pt'], row['fra_rel_enroute']) 
                     for _, row in fra_df_subset.iterrows()]

        # Rename fra_points using fra_map if any points were renamed
        # fra_points = [(fra_map.get(point, point), type_) 
        #              for point, type_ in fra_points]

        # Connect points according to rules
        for point1, type1 in tqdm(fra_points, desc=f"{fra}"):
            for point2, type2 in fra_points:
                # Skip self-connections
                if point1 == point2:
                    continue
                    
                # Rule 2: EX can connect to X, EX or I
                if type1 == 'EX' and type2 in ['X', 'EX', 'I']:
                    create_edge_with_closest_refs(ats_graph, point1, point2, threshold=150, type='FRA')
                
                # Rule 3: E can connect to X, EX or I  
                elif type1 == 'E' and type2 in ['X', 'EX', 'I']:
                    create_edge_with_closest_refs(ats_graph, point1, point2, threshold=150, type='FRA')
                
                # Rule 4: I can connect bilaterally to I
                elif type1 == 'I' and type2 == 'I':
                    create_edge_with_closest_refs(ats_graph, point1, point2, threshold=150, type='FRA')
    
    print(f'FRA graph merged into ATS graph. Nodes: {ats_graph.number_of_nodes()}, edges: {ats_graph.number_of_edges()}')
    return ats_graph

In [ ]:
ats_fra = load_fra_graph_into_ats_graph(ats_graph, fra_df)

In [ ]:
# Save the ats_fra graph to a file in GraphML format (most compact format for NetworkX graphs)
import os
from pathlib import Path

# Create output directory if it doesn't exist
output_dir = Path(os.getenv('PROJECT_ROOT', '.')) / 'data' / 'graphs'
output_dir.mkdir(parents=True, exist_ok=True)

# Save the graph
output_path = output_dir / 'ats_fra_graph.gml'
import networkx as nx
nx.write_gml(ats_fra, output_path)

print(f"Graph saved to {output_path}")

In [ ]:
# Get the size of the graph saved
print(f"Graph size: {os.path.getsize(output_path)} bytes or {os.path.getsize(output_path) / 1024 / 1024:.2f} MB")

# Build an edge free graph

In [ ]:
import time, os
time_start = time.time()
print('Reading navigation graph...')
import pickle
G = pickle.load(open(os.path.join(PROJECT_ROOT, 'data', 'graphs', 'ats_fra_graph.pkl'), 'rb'))
print(f'Navigation graph loaded in {time.time() - time_start:.2f} seconds')

In [ ]:
import networkx as nx
import pandas as pd
# Create a new graph with only the nodes from G, but no edges
print('Creating a new graph with only nodes (no edges)...')
start_time = time.time()

# Create an empty graph with the same graph type as G
node_only_graph = nx.Graph() if isinstance(G, nx.Graph) else nx.DiGraph()

# Add all nodes from G with their attributes
for node, attrs in G.nodes(data=True):
    node_only_graph.add_node(node, **attrs)

# Load airports data and add them to the graph
print('Loading airports data...')
airports_path = os.path.join(PROJECT_ROOT, 'data', 'airac', 'airports.csv')
if os.path.exists(airports_path):
    airports_df = pd.read_csv(airports_path)
    print(f'Adding {len(airports_df)} airports to the graph...')
    
    # Add airports as nodes to the graph
    for _, airport in airports_df.iterrows():
        if pd.notna(airport['icao']) and pd.notna(airport['latitude']) and pd.notna(airport['longitude']):
            node_only_graph.add_node(
                airport['icao'],
                lat=airport['latitude'],
                lon=airport['longitude'],
                name=airport['name'] if pd.notna(airport['name']) else "",
                elevation=airport['elevation'] if pd.notna(airport['elevation']) else 0,
                type="airport"
            )
else:
    print('Airports data file not found.')

print(f'Node-only graph created in {time.time() - start_time:.2f} seconds')
print(f'Number of nodes: {node_only_graph.number_of_nodes()}')
print(f'Number of edges: {node_only_graph.number_of_edges()}')

# Save the node-only graph
node_graph_path = os.path.join(PROJECT_ROOT, 'data', 'graphs', 'ats_fra_nodes_only.gml')
nx.write_gml(node_only_graph, node_graph_path)
print(f"Node-only graph saved to {node_graph_path}")
print(f"Node-only graph size: {os.path.getsize(node_graph_path) / 1024 / 1024:.2f} MB")


# Build the predecessor and successor into the graph

To amortize the navigation cost

In [ ]:
raise Exception('Do not proceed further! There are too many edges to be processed!')

In [ ]:
import time, os
time_start = time.time()
print('Reading navigation graph...')
import pickle
G = pickle.load(open(os.path.join(PROJECT_ROOT, 'data', 'graphs', 'ats_fra_graph.pkl'), 'rb'))
print(f'Navigation graph loaded in {time.time() - time_start:.2f} seconds')

In [3]:
num_edges = len(G.edges())
edges = list(G.edges())

Single threaded

In [ ]:
from tqdm import tqdm
# Precompute edge successors and predecessors
edge_successors = {}
edge_predecessors = {j: [] for j in range(num_edges)}

# First create an adjacency mapping from nodes to edge indices
node_to_outgoing_edges = {}
for k, (u, v) in tqdm(enumerate(edges), total=num_edges, desc='Building edge successors and predecessors'):
    if v not in node_to_outgoing_edges:
        node_to_outgoing_edges[v] = []
    if u not in node_to_outgoing_edges:
        node_to_outgoing_edges[u] = []
    node_to_outgoing_edges[v].append(k)

# Now build the edge successors and predecessors in O(E) time
for k, (u, v) in tqdm(enumerate(edges), total=num_edges, desc='Building edge successors and predecessors'):
    # Include self (staying on same edge)
    successors = [k]
    # Add all edges starting with v
    if v in node_to_outgoing_edges:
        for j in node_to_outgoing_edges[v]:
            if j != k:  # Avoid adding self twice
                successors.append(j)
                edge_predecessors[j].append(k)
    
    # Add self as predecessor (staying on same edge)
    edge_predecessors[k].append(k)
    edge_successors[k] = successors

Multi-threaded

In [ ]:
# Save the edge successors and predecessors
pickle.dump((edge_successors, edge_predecessors), open(os.path.join(PROJECT_ROOT, 'data', 'graphs', 'ats_fra_pred_succ.pkl'), 'wb'))
